In [1]:
import pandas as pd

In [ ]:
sales = pd.read_csv("sales.csv")

In [ ]:
sales

,Week,Product,Sales,Promotion,Holiday
0,1,Product_1,152.0,1,0
1,2,Product_1,485.0,0,0
2,3,Product_1,398.0,0,0
3,4,Product_1,320.0,0,0
4,5,Product_1,156.0,0,0
5,6,Product_1,121.0,1,0
6,7,Product_1,238.0,0,0
7,8,Product_1,70.0,1,0
8,9,Product_1,152.0,1,0
9,10,Product_1,171.0,0,0


In [ ]:
sales.shape

(30, 5)

In [21]:
def process_sales(sales: pd.DataFrame) -> pd.DataFrame:
    sales_transformed = sales.copy()
    sales_transformed.index = pd.to_datetime(sales_transformed["Week"], unit="W")
    sales_transformed.index.name = "Week Date"
    # sales_transformed.dropna(inplace=True)
    sales_transformed["Sales Lag_1"] = sales_transformed["Sales"].shift(1)
    sales_transformed["Sales Lag_2"] = sales_transformed["Sales"].shift(2)

    train_sarimax = sales_transformed[sales_transformed.index <= "1970-07-16"]
    train_other_models = train_sarimax.dropna()
    train_sarimax = train_sarimax.drop(columns=["Sales Lag_1", "Sales Lag_2"])
    test_sales = sales_transformed[sales_transformed.index > "1970-07-16"]

    return train_sarimax, train_other_models, test_sales

In [22]:
train_sarimax, train_other_models, test_sales = process_sales(sales)

In [25]:
# train_sarimax
# train_other_models
test_sales

,Week,Product,Sales,Promotion,Holiday,Sales Lag_1,Sales Lag_2
Week Date,,,,,,,
1970-07-23,29,Product_1,NaN,1,0,326.0,493.0
1970-07-30,30,Product_1,NaN,0,1,NaN,326.0


# SARIMAX

In [10]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

In [27]:
sarimax_model = SARIMAX(
    train_sarimax["Sales"],
    exog=train_sarimax[["Promotion", "Holiday"]],
    order=(1, 1, 1),
    seasonal_order=(1, 1, 1, 4),
)
sarimax_results = sarimax_model.fit()


/home/carroyo/personal_projects/mercadolibre_test/.venv/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-THU will be used.
  self._init_dates(dates, freq)
/home/carroyo/personal_projects/mercadolibre_test/.venv/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-THU will be used.
  self._init_dates(dates, freq)


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            7     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  5.28689D+00    |proj g|=  1.27856D-01

At iterate    5    f=  5.25612D+00    |proj g|=  1.88829D-02

At iterate   10    f=  5.25429D+00    |proj g|=  2.42762D-03


 This problem is unconstrained.



At iterate   15    f=  5.25185D+00    |proj g|=  2.91539D-02

At iterate   20    f=  5.19747D+00    |proj g|=  9.79558D-03

At iterate   25    f=  5.19580D+00    |proj g|=  1.51243D-03

At iterate   30    f=  5.19163D+00    |proj g|=  3.52812D-03

At iterate   35    f=  5.16017D+00    |proj g|=  3.28222D-02

At iterate   40    f=  5.15160D+00    |proj g|=  3.71015D-04

At iterate   45    f=  5.15099D+00    |proj g|=  1.76847D-02

At iterate   50    f=  5.14560D+00    |proj g|=  6.98797D-03

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    7     50     56      1     0     0   6.988D-03   5.146D+00
  F =   5.14560091322

/home/carroyo/personal_projects/mercadolibre_test/.venv/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [28]:
sarimax_forecast = sarimax_results.forecast(
    steps=len(test_sales),
    exog=test_sales[['Promotion', 'Holiday']]
)

In [29]:
sarimax_forecast

1970-07-23    360.766767
1970-07-30    344.749084
Freq: W-THU, Name: predicted_mean, dtype: float64